In [ ]:
# install packages
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install h2o

In [1]:
# import packages
import os
import numpy as np
import pandas as pd

# H2O
import h2o
from h2o.automl import H2OAutoML

import catboost
import lightgbm
import xgboost

from sklearn.preprocessing import LabelEncoder
from joblib import load, dump

C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Chen\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [1]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS May"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - May 2021'
os.chdir(path)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
# Initialize and Connect to H2O
h2o.init()

hof_train = h2o.H2OFrame(df_train.copy())
hof_test = h2o.H2OFrame(df_test.copy())

feature_col = [i for i in df_train.columns if i not in ['id', 'target']]
target_col = 'target'

# for binary or multiclass
hof_train[target_col] = hof_train[target_col].asfactor()

# Warnings : Xgboost is not supported on windows
h2o.estimators.xgboost.H2OXGBoostEstimator.available()

In [ ]:
# construct model
PROJECT_NAME = 'h2o-20210522'
automl = H2OAutoML(project_name = PROJECT_NAME,
                   seed=42,
                   max_runtime_secs=8*60*60,
                   nfolds = 5,
                   balance_classes = True,
                   # include_algos = ['DRF', 'GLM', 'XGBoost', 'GBM', "DeepLearning", 'StackedEnsemble'],
                   exclude_algos = ["DeepLearning"],
                   )

In [ ]:
# training model
automl.train(x=feature_col, 
             y=target_col, 
             training_frame=hof_train)

In [ ]:
# Save all model
k = len(automl.leaderboard)
for i in range(k):
   aml = h2o.get_model(automl.leaderboard[i, 0])
   h2o.save_model(model=aml, path=PROJECT_NAME + '/ALL_MODEL', force=True)

In [ ]:
# save best model
best_model_path = h2o.save_model(model=automl.leader, path=PROJECT_NAME, force=True)

In [ ]:
# load best model
best_model = h2o.load_model(best_model_path)

In [ ]:
# predict probability
result = best_model.predict(hof_test).as_data_frame()
print(result.head())

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result.iloc[:, 1:]
sub.to_csv(f'{PROJECT_NAME}.csv', index=False)

In [ ]:
# detail
lb = automl.leaderboard 
print(lb.as_data_frame())

In [ ]:
# 關閉H2O
h2o.shutdown()